In [1]:
import pandas as pd
import math
import folium

In [2]:
df_supermarket_products = pd.read_csv('../../data/aryan_pandas/supermarket_products.csv',encoding='cp1252')
df_all_supermarket_location = pd.read_csv('../../data/aryan_pandas/establishment_catalonia.csv')
df_customer_info = pd.read_csv('../../data/aryan_pandas/customers.csv')
df_cust_location = pd.read_csv('../../data/aryan_pandas/location.csv')
df_cust_loc_mapping = pd.read_csv('../../data/aryan_pandas/customer_location.csv')

In [3]:
df_all_supermarket_location.head()

,id,commercial_name,county_code,full_address,UTMx,UTMy,latitude,longitude
0,cens-0828450006-2602,AREA OPTICA SLL,21.0,"C/ CAMI RAL 119, 08490 TORDERA",476705.75,4616135.15,41.696607,2.720054
1,cens-0828450006-2852,FEDEROPTICS TORDERA -ESPAI OPTIC LLORET,21.0,"c/ Amadeu Vives 25, 08490 TORDERA",476806.97,4616300.60,41.698100,2.721264
2,cens-0827320002-668,ANTIGA ESCOLA DE CA L'AVI,3.0,"PL. SANT JOAN 1, 08739 SUBIRATS",394544.00,4585057.00,41.410093,1.738247
3,cens-0804780001-587,ÒPTICA JANÉ,7.0,"PL DE LA FIRA 6, B, 08261 Cardona",390568.98,4640996.16,41.913291,1.680453
4,cens-0811410007-2891,ÒPTICA UNIVERSITÀRIA,11.0,"RBLA BOBILES 1, LOC3, 08760 MARTORELL",409650.86,4592277.96,41.476965,1.917883


### Plotting all the supermarkets

In [4]:
df_plot = df_all_supermarket_location.sample(n=50, random_state=42)

# Initialize the map centered around the mean latitude and longitude
m = folium.Map(location=[df_plot['latitude'].mean(), df_plot['longitude'].mean()], zoom_start=13)

# Add points to the map
for _, row in df_plot.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['commercial_name']
    ).add_to(m)

m

In [4]:
# Products available in different stores

df_supermarket_products.head()

,store_id,store_name,product_id,product_name,product_price,manufacture_date,expiry_date,quantity
0,cens-0820210007-3653,ESCLAT-BON PREU,105.1.1,Amul Kool (Kesar),23,01 Mar 2024,28 Aug 2024,21
1,cens-0820210007-3653,ESCLAT-BON PREU,46.1.9,PARLE Krack Jack Crackers Sweet & Salty (Mitha...,4,01 Feb 2024,27 Sep 2024,54
2,cens-0820210007-3653,ESCLAT-BON PREU,33.1.10,RUNutz Almonds (Badam) (1 kg),"1,025",01 Feb 2024,29 Jul 2024,7
3,cens-0820210007-3653,ESCLAT-BON PREU,28.1.17,"Utkal's Kasper HMT Rice (Full Grain, Steam) (...",826,01 Feb 2024,27 Oct 2024,68
4,cens-0820210007-3653,ESCLAT-BON PREU,58.1.15,"Nescafe Classic Coffee Powder, 100% Pure Insta...",17,01 Jul 2023,29 Jan 2025,15


In [5]:
df_supermarkets = pd.merge(df_supermarket_products, df_all_supermarket_location, left_on='store_id', right_on='id', how='left')
df_supermarkets.head()

,store_id,store_name,product_id,product_name,product_price,manufacture_date,expiry_date,quantity,id,commercial_name,county_code,full_address,UTMx,UTMy,latitude,longitude
0,cens-0820210007-3653,ESCLAT-BON PREU,105.1.1,Amul Kool (Kesar),23,01 Mar 2024,28 Aug 2024,21,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811
1,cens-0820210007-3653,ESCLAT-BON PREU,46.1.9,PARLE Krack Jack Crackers Sweet & Salty (Mitha...,4,01 Feb 2024,27 Sep 2024,54,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811
2,cens-0820210007-3653,ESCLAT-BON PREU,33.1.10,RUNutz Almonds (Badam) (1 kg),"1,025",01 Feb 2024,29 Jul 2024,7,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811
3,cens-0820210007-3653,ESCLAT-BON PREU,28.1.17,"Utkal's Kasper HMT Rice (Full Grain, Steam) (...",826,01 Feb 2024,27 Oct 2024,68,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811
4,cens-0820210007-3653,ESCLAT-BON PREU,58.1.15,"Nescafe Classic Coffee Powder, 100% Pure Insta...",17,01 Jul 2023,29 Jan 2025,15,cens-0820210007-3653,ESCLAT-BON PREU,41.0,"C AMADEU VIVES cant Industria, 08470 Sant Celoni",457873.4,4615087.79,41.686399,2.493811


In [6]:
# just for the sake of the query - removing the same locations
df_supermarkets = df_supermarkets.drop_duplicates(subset=['latitude', 'longitude'])
print(len(df_supermarkets))

df_supermarkets['county_code'].value_counts()

318


county_code
21.0    88
41.0    67
24.0    35
7.0     34
11.0    24
6.0     19
14.0    15
17.0    13
40.0     9
42.0     7
3.0      7
Name: count, dtype: int64

### Plotting all Supermarkets having products

In [14]:
# sampling 10 out of all the supermarkets having the same county code
df_plot = df_supermarkets.query('county_code == 21.0')
df_plot = df_plot.sample(n=40, random_state=42)

# Initialize the map centered around the mean latitude and longitude
m = folium.Map(location=[df_plot['latitude'].mean(), df_plot['longitude'].mean()], zoom_start=13)

# Add points to the map
for _, row in df_plot.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['store_name']
    ).add_to(m)

m

In [12]:
len(df_plot)

30

### Plotting Customers (with unique locations)

In [36]:
df_customer = df_customer_info.merge(df_cust_loc_mapping, on='customer_id')
df_customer = df_customer.merge(df_cust_location, on='location_id')
# just for the sake of the query - removing the same locations
df_customer = df_customer.drop_duplicates(subset=['latitude', 'longitude'])
df_customer.reset_index(drop=True, inplace=True)
print(len(df_customer))


263


In [37]:
df_customer.head()

,customer_id,customer_name,email_id,location_id,postal_code,latitude,longitude
0,1,Debra Kelly,sony.sth8@gmail.com,21788,8940,41.3500,2.0833
1,2,Nicholas Cardenas,sony.sth8@gmail.com,21308,8193,41.5026,2.0875
2,3,Barbara Holder,sony.sth8@gmail.com,21598,8630,41.5194,1.9015
3,4,Jermaine Johnson,sony.shrestha@estudiantat.upc.edu,21500,8472,41.7247,2.4653
4,5,Joyce Austin MD,sony.sth8@gmail.com,21599,8635,41.4945,1.8722


In [38]:
df_customer.query('postal_code == 8490')  # Postal code of county code 21 (Ajuntament of Tordera)

,customer_id,customer_name,email_id,location_id,postal_code,latitude,longitude
95,121,Michael Roth,sony.sth8@gmail.com,21508,8490,41.6975,2.7179


In [32]:
# sampling out of all the customers
df_plot = df_customer.sample(n=40, random_state=42)

# Initialize the map centered around the mean latitude and longitude
m = folium.Map(location=[df_plot['latitude'].mean(), df_plot['longitude'].mean()], zoom_start=13)

# Add points to the map
for _, row in df_plot.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['customer_name']
    ).add_to(m)

m

In [9]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [39]:
# choose customer
customer_index = 95
df_chosen_customer =  df_customer.iloc[[customer_index]]
df_chosen_customer
# chosen_cust_location = {"lat": df_customer.loc[customer_index]['latitude'], "lon": df_customer.loc[customer_index]['longitude']}

,customer_id,customer_name,email_id,location_id,postal_code,latitude,longitude
95,121,Michael Roth,sony.sth8@gmail.com,21508,8490,41.6975,2.7179


In [40]:
# Calculate distances and store in the DataFrame from the customer
df_supermarkets['distance_from_customer'] = df_supermarkets.apply(
    lambda row: haversine(df_chosen_customer['latitude'], 
                          df_chosen_customer['longitude'], 
                          row['latitude'], row['longitude']),
    axis=1
)

C:\Users\Aryan Gupta\AppData\Local\Temp\ipykernel_6468\3056038853.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dlat = math.radians(lat2 - lat1)
C:\Users\Aryan Gupta\AppData\Local\Temp\ipykernel_6468\3056038853.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dlon = math.radians(lon2 - lon1)
C:\Users\Aryan Gupta\AppData\Local\Temp\ipykernel_6468\3056038853.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2


In [41]:
# Sort the DataFrame by distance and select the top 5 closest supermarkets
top_5_closest_supermarkets = df_supermarkets.sort_values(by='distance_from_customer').head(5)
top_5_closest_supermarkets

,store_id,store_name,product_id,product_name,product_price,manufacture_date,expiry_date,quantity,id,commercial_name,county_code,full_address,UTMx,UTMy,latitude,longitude,distance_from_customer
12600,cens-0828450006-166,DIA,87.1.15,Cadbury Perk Chocolate coated wafer Home Treat...,64,01 Jan 2024,30 Sep 2024,85,cens-0828450006-166,DIA,21.0,"C CAMI RAL 113, 08490 TORDERA",476698.13,4616171.18,41.696931,2.719961,0.182455
1480,cens-0828450006-2752,CONSUM SOCIETAT COOPERATIVA VALENCIANA,94.1.6,DNV Mixed Pickle (Acharo) (1 kg),220,01 Feb 2024,25 Jan 2025,96,cens-0828450006-2752,CONSUM SOCIETAT COOPERATIVA VALENCIANA,21.0,"Av. Fibracolor 15-17, 08490 TORDERA",476903.53,4616095.29,41.696254,2.722433,0.401043
8440,cens-0828450006-2849,MERCADONA,1.1.1,Tata Sampann Toor/Arhar Dal (Harada Dali) (Spl...,135,01 Feb 2024,25 Jan 2025,15,cens-0828450006-2849,MERCADONA,21.0,"Av. Fibracolor s/núm, 08490 TORDERA",477015.30,4615936.76,41.694829,2.723782,0.571587
10440,cens-0826130008-549,SUPERMERCAT CARME,33.1.1,Happilo Essentials Californian Popular Almonds...,625,01 Oct 2023,30 Sep 2024,95,cens-0826130008-549,SUPERMERCAT CARME,21.0,"AV TORRENTO DE CAN GELAT LOCAL 3, 08398 Santa ...",476960.00,4609406.00,41.636003,2.723369,6.853185
10840,cens-0826130008-117,CLAREL,87.1.32,GALAXY Smooth Milk Chocolate Bars (56 g),90,01 Dec 2023,24 Nov 2024,36,cens-0826130008-117,CLAREL,21.0,"C MONTAGUT 13-15, 08398 Santa Susanna",475637.00,4609403.00,41.635937,2.707485,6.899932


In [42]:
# Plot the closest supermarkets with products & the customer

# this is not correct -centre around the customer
map_center = [(top_5_closest_supermarkets['latitude'].mean() + top_5_closest_supermarkets['latitude'].mean()) / 2,
              (df_chosen_customer['longitude'].mean() + df_chosen_customer['longitude'].mean()) / 2]

# Initialize the map centered around the mean latitude and longitude
mymap = folium.Map(location=map_center, zoom_start=2)

def add_marker_supermarket(df, color):
    for _, row in df.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=row['store_name'],
            icon=folium.Icon(color=color)
        ).add_to(mymap)
        
def add_marker_customer(df, color):
    for _, row in df.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=row['customer_name'],
            icon=folium.Icon(color=color)
        ).add_to(mymap)
        
add_marker_supermarket(top_5_closest_supermarkets, 'blue')
add_marker_customer(df_chosen_customer, 'red')

mymap

In [71]:
# Plot the supermarkets and the customer and connect the customer to the supermarket and mention the distance

# Initialize the map centered around the customer
m = folium.Map(location=[df_chosen_customer['latitude'].mean(), df_chosen_customer['longitude'].mean()], zoom_start=13)

for _, customer in df_chosen_customer.iterrows():
    folium.Marker(
        location=[customer['latitude'], customer['longitude']],
        popup=customer['customer_name'],
        icon=folium.Icon(color='red')
    ).add_to(m)
    

# Function to add a supermarket marker and a line between the customer and the supermarket with distance popup
def add_supermarket_and_line(m, customer, supermarket):
    # Add supermarket marker
    folium.Marker(
        location=[supermarket['latitude'], supermarket['longitude']],
        popup=supermarket['store_name'] + '\n' + str(supermarket['distance_from_customer']),
        icon=folium.Icon(color='blue')
    ).add_to(m)
    
    # Add a line between the customer and the supermarket
    line = folium.PolyLine(
        locations=[
            [customer['latitude'], customer['longitude']],
            [supermarket['latitude'], supermarket['longitude']]
        ],
        color='blue',
        dash_array = '10'
    ).add_to(m)

# Add supermarkets and lines to the map
for _, supermarket in top_5_closest_supermarkets.iterrows():
    add_supermarket_and_line(m, df_chosen_customer.iloc[0], supermarket)
    
# Add a line between the customer and the closest supermarket
    line = folium.PolyLine(
        locations=[
            [df_chosen_customer['latitude'].iloc[0], df_chosen_customer['longitude'].iloc[0]],
            [top_5_closest_supermarkets['latitude'].iloc[0], top_5_closest_supermarkets['longitude'].iloc[0]]
        ],
        color='green',
    ).add_to(m)

m